In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 9

In [8]:
path_0 = '../Data/PEMS-BAY/speed_bay_0.csv'
path_5 = '../Data/PEMS-BAY/speed_bay_5.csv'
path_10 = '../Data/PEMS-BAY/speed_bay_10.csv'
path_20 = '../Data/PEMS-BAY/speed_bay_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(52116, 325) 0


((3380975, 9), 0)

### Unobserved Node = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(52116, 325) 1667712
Node 0
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 66.68035124937323 66.57635617930714 1.0 0.0 0.14687633514404297
Node 1
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.002 )
( 15.295689628449892 8.220059352996374 0.12285702741103648 0.770815042278983 320.38468623161316
Node 2
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.005 )
( 12.295152975348282 7.832662719585675 0.11750723495554788 0.8153624511169215 885.8100252151489
Node 3
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.01 )
( 10.016385561397366 6.326934081303493 0.10077496383134915 0.8472090783984898 793.5127341747284
Node 4
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.012 )
( 13.55576734474368 9.877784128326162 0.18886363637640805 0.7764260254257975 844.6657221317291
Node 5
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 62.63560231965087 62.28564837066228 1.0 0.0 0.17903900146484375
Node 6
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.002 )
( 15.80106131533529 9.998

( 22.773949531623636 16.29898130461548 0.557160927859338 0.58558415019164 626.1981151103973
Node 53
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 60.11477823738545 58.34264581797988 1.0 0.0 0.1781911849975586
Node 54
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.0 )
( 10.991522560132232 5.627920164355936 0.08472090412837054 0.8357238727036508 713.3569443225861
Node 55
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 66.62074929718848 66.55998269729872 1.0 0.0 0.17793846130371094
Node 56
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 16.680516729367696 9.558430383925144 0.14643398243633543 0.7487737558100549 293.8034756183624
Node 57
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 64.27975448207522 63.70782573402981 1.0 0.0 0.17158770561218262
Node 58
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 55.64853378749757 53.67795294092514 1.0 0.0 0.1778113842010498
Node 59
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 61.63632419536

( 15.280522948199483 10.58753467591291 0.20530352901627472 0.7502665902553655 392.36843395233154
Node 106
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.003 )
( 22.088678436069216 15.473654186881769 0.5117991733496808 0.5949212365511873 416.85957622528076
Node 107
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.004 )
( 19.61291131790929 12.587099553563288 0.3506720204543196 0.676143363931498 658.005998134613
Node 108
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.001 )
( 12.000907925402784 5.358341666175927 0.08111328821136404 0.8198696936116545 435.9205460548401
Node 109
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.008 )
( 15.629902909069258 8.42857359810956 0.12814903595146435 0.7650852039410805 443.0431146621704
Node 110
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.004 )
( 5.485103438705676 3.3156699422296465 0.051656638376307844 0.9174476309752431 958.9460823535919
Node 111
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.002 )
( 10.336783464179774 5.84557090124

( 8.041681117661787 6.061667295360879 0.10495289210933287 0.869585453856143 1071.8968472480774
Node 159
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.021 )
( 9.2272719172121 5.390537501045563 0.11539553939823342 0.8480886403364258 1232.036107301712
Node 160
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 54.230408878798 51.774547940230754 1.0 0.0 0.17748355865478516
Node 161
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.012 )
( 13.121936282938256 7.851406348640085 0.18734276965237928 0.7903925253313748 579.1945600509644
Node 162
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.003 )
( 13.184919499226769 8.568744871221817 0.12777608011103092 0.8022796589831926 597.5014340877533
Node 163
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.016 )
( 10.990264918261548 7.058955969326191 0.10617474901155635 0.8348563754981347 689.2644898891449
Node 164
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 3.5683974023486598 2.6039083189232235 0.039425216036502425 0.9463

( 7.976904392754421 5.621465980919402 0.08372364331431204 0.8805737736632941 992.7535603046417
Node 211
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.128 )
( 9.695258335794954 6.614744202507399 0.11243098716993132 0.8537725186878729 594.5385401248932
Node 212
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.043 )
( 6.961094110174707 3.8374114552705234 0.10818005394606472 0.8914978502785337 1327.1232562065125
Node 213
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.124 )
( 9.839800245640435 5.459799139386707 0.11171062028181059 0.8454166168252083 711.2849645614624
Node 214
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.007 )
( 10.901903883517337 7.007652254086221 0.13855351500326826 0.8303172613850679 802.8393852710724
Node 215
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.015 )
( 6.173028399303356 4.49823641120145 0.07328216837048183 0.9051011287781581 1372.7974617481232
Node 216
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.0 )
( 6.9047218124378364 3.818189284487855

( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.008 )
( 4.5052359403301105 3.050460070646596 0.04645214915437516 0.9322273300170942 1446.614726781845
Node 264
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 66.58330360701665 66.51366379356415 1.0 0.0 0.1787106990814209
Node 265
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.045 )
( 5.933138651764157 3.728963033095667 0.06796845419257805 0.9089101008244921 1152.2506806850433
Node 266
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.056 )
( 7.925891008745617 4.8602734985051 0.11427558650945718 0.8744665529931512 1166.6655957698822
Node 267
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.004 )
( 8.629562054851261 5.539378531545254 0.11285765105983228 0.8616680789799664 1045.8838617801666
Node 268
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.029 )
( 8.460228940419295 5.876638167336129 0.09722826836390618 0.8672132502972574 1234.0584931373596
Node 269
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.018 )
( 6.594495

( 6.770397202117159 3.8338705151928867 0.05845280937035202 0.8981569586062063 893.0035307407379
Node 317
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.02 )
( 9.092694010568218 4.8083019937033775 0.07253050435343361 0.8630651204516189 828.6853468418121
Node 318
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.003 )
( 7.554529490893114 5.823727301002854 0.09478084308185264 0.8832820224074031 1248.4325034618378
Node 319
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.015 )
( 11.92267377339927 8.238193821066988 0.18510837731172128 0.8007266880305427 1416.6277985572815
Node 320
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.008 )
( 12.745964678749093 7.93923535085108 0.144566629710828 0.792301894808765 891.2862877845764
Node 321
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.052 )
( 15.847928804221157 10.89678214305627 0.2863498316830224 0.7310330032316705 981.8747613430023
Node 322
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.004 )
( 18.58999900202057 12.24457851609792

### Unobserved Node = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(52116, 325) 3387540
Node 0
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.004 )
( 7.814306200976785 5.111519658182142 0.08250572824942264 0.882809462539383 787.4035568237305
Node 1
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.002 )
( 15.295689628449892 8.220059352996374 0.12285702741103648 0.770815042278983 317.29418087005615
Node 2
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.005 )
( 12.295152975348282 7.832662719585675 0.11750723495554788 0.8153624511169215 876.9204504489899
Node 3
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.01 )
( 10.016385561397366 6.326934081303493 0.10077496383134915 0.8472090783984898 785.3642358779907
Node 4
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.012 )
( 13.55576734474368 9.877784128326162 0.18886363637640805 0.7764260254257975 838.0311546325684
Node 5
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.014 )
( 11.902737856993193 6.862106196240442 0.1304862435216088 0.809968493697092 606.9859268665314
Node 6
( (41671, 12) (41671

( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.001 )
( 20.512734005820842 10.855977571522606 0.16335116338916927 0.6920969184192812 443.1413552761078
Node 56
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 16.680516729367696 9.558430383925144 0.14643398243633543 0.7487737558100549 292.22424602508545
Node 57
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.093 )
( 12.982226926468119 7.321011395921469 0.1483069602356798 0.7980355240764292 1150.5460114479065
Node 58
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 55.64853378749757 53.67795294092514 1.0 0.0 0.17845630645751953
Node 59
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 14.624209296535112 9.198809885463527 0.17713175919177646 0.7627339156341962 552.3817076683044
Node 60
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.002 )
( 15.040740455483759 9.221184857044078 0.19301081680285354 0.7534759973479731 462.49075531959534
Node 61
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.008 )
( 22.9482

( 15.629902909069258 8.42857359810956 0.12814903595146435 0.7650852039410805 440.5417718887329
Node 110
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 66.44392527440041 66.37906373161576 1.0 0.0 0.17441058158874512
Node 111
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.002 )
( 10.336783464179774 5.84557090124911 0.1435560087888683 0.838467688787823 858.5700953006744
Node 112
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.027 )
( 24.05247864406832 17.335020198421724 0.64926595832772 0.553223376961262 520.0279316902161
Node 113
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.005 )
( 16.34638921616246 9.583953778636786 0.20465687775312416 0.7325394040448356 433.3795244693756
Node 114
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.004 )
( 16.386335344892547 10.025669656580071 0.20171439258240412 0.7318310495176236 335.7600128650665
Node 115
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.171 )
( 20.247462119428786 14.942126607049419 0.4804146715646573 0.625278113646

( 3.5683974023486598 2.6039083189232235 0.039425216036502425 0.9463985149641371 1355.4837584495544
Node 165
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.107 )
( 14.40926634610739 8.944374628530333 0.18311773828071978 0.7744007610669986 335.2034578323364
Node 166
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.057 )
( 22.491016365770275 15.645715361287232 0.6319058159761226 0.5831790489035846 979.478985786438
Node 167
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.002 )
( 9.803946981176418 6.242376782006483 0.14320309819076632 0.839327272055856 890.7954232692719
Node 168
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.007 )
( 10.499528339586465 7.177662914070165 0.14989417677840658 0.828638898661498 1082.4698448181152
Node 169
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.004 )
( 21.01589811286296 14.786560883019904 0.4916677888046856 0.6181923940702161 495.26323342323303
Node 170
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.014 )
( 14.944104840415704 8.42864923

( 14.669890222084971 7.24914077508183 0.10935953582772077 0.7794370461127362 436.4060261249542
Node 218
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.035 )
( 9.045659752440587 5.5035292338226505 0.08265983307779254 0.8635727649447462 909.6438837051392
Node 219
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.022 )
( 13.644113452313599 8.115342196397197 0.16351008738882236 0.7833915887854678 641.5494878292084
Node 220
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.013 )
( 21.170411257745908 15.18087108858603 0.5009019603323135 0.61013886443347 592.1709771156311
Node 221
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.003 )
( 9.762127392504365 6.137946782004702 0.13520488879911552 0.83784259153923 877.274783372879
Node 222
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.078 )
( 10.75319198829954 6.687243007641289 0.13698949436406194 0.822380596051874 513.2397012710571
Node 223
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 17.414744316701572 11.27900295929026 0.43

( 6.164436389483375 3.6155473564942238 0.05481246396460197 0.9075926820237745 943.2971088886261
Node 272
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.103 )
( 5.430453286044599 3.9561580226312083 0.05925710230973572 0.9182813464641483 1251.1163263320923
Node 273
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 5.374316640121036 4.264349777462618 0.06365316590867375 0.919328213253957 1292.9217722415924
Node 274
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.012 )
( 14.266069184998342 8.11812250080637 0.12591748293376803 0.7842008330433246 693.2341251373291
Node 275
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.021 )
( 14.898620493134331 9.048757420462438 0.15343380461019127 0.767334383986354 797.8702208995819
Node 276
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.004 )
( 14.350579521509303 8.278137723062736 0.15145654410480494 0.7717681539463379 531.2075042724609
Node 277
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) -0.001 )
( 11.065556005656797 6.54444763876

( 10.774553247976497 6.594565977707966 0.13594372219359505 0.8318439004664409 877.6529557704926
Node 324
( (41671, 12) (41671, 9) (10403, 12) (10403, 9) 0.0 )
( 66.52010593880324 66.42106123233685 1.0 0.0 0.1777496337890625
RMSE: 22.84637732208845, MAE: 19.036850329042718, MAPE: 0.34873544572060433, ACC: 0.6356044054062668
